In [ ]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

# obj1

In [9]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
FILE_OPTION = "obj_1"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 100
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100


### test

In [10]:
I_START = 0
N_DAT = 50-I_START

In [11]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 10.4    5.5     5.9     13.0    -------
========== 1 / 50 (eta: 34.9 / 1742.9 s) =========
------- 41.9    17.2    6.5     7.0     -------
========== 2 / 50 (eta: 107.5 / 2688.3 s) =========
------- 58.9    6.2     5.5     6.2     -------
========== 3 / 50 (eta: 184.3 / 3072.5 s) =========
------- 27.0    5.6     5.5     5.9     -------
========== 4 / 50 (eta: 228.5 / 2855.8 s) =========
------- 5.9     5.9     6.4     6.6     -------
========== 5 / 50 (eta: 253.2 / 2531.6 s) =========
------- 30.2    6.8     6.1     6.4     -------
========== 6 / 50 (eta: 302.7 / 2522.4 s) =========
------- 8.3     7.6     5.6     6.0     -------
========== 7 / 50 (eta: 330.2 / 2358.4 s) =========
------- 38.6    11.9    6.6     105.5   -------
========== 8 / 50 (eta: 492.8 / 3079.7 s) =========
------- 9.6     5.2     5.7     9.4     -------
========== 9 / 50 (eta: 522.6 / 2903.1 s) =========
------- 118.7   5.3     6.5     6.2     -------
========== 10 / 50 (eta: 659.3 / 3296.7 s) =========


# obj3

In [6]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [7]:
I_START=0
N_DAT = 50-I_START

### test

In [8]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 16.2    5.0     6.7     6.1     -------
========== 1 / 50 (eta: 34.0 / 1699.7 s) =========
------- 6.8     19.3    8.1     6.5     -------
========== 2 / 50 (eta: 74.7 / 1867.7 s) =========
------- 99.8    12.4    22.8    6.6     -------
========== 3 / 50 (eta: 216.3 / 3605.6 s) =========
------- 104.8   25.2    8.0     55.8    -------
========== 4 / 50 (eta: 410.2 / 5127.2 s) =========
------- 91.9    6.9     6.7     5.9     -------
========== 5 / 50 (eta: 521.6 / 5215.6 s) =========
------- 34.3    5.1     7.2     8.0     -------
========== 6 / 50 (eta: 576.2 / 4801.3 s) =========
------- 7.5     7.1     6.0     6.3     -------
========== 7 / 50 (eta: 603.0 / 4307.1 s) =========
------- 22.0    6.3     5.9     5.8     -------
========== 8 / 50 (eta: 643.1 / 4019.2 s) =========
------- 67.5    5.5     5.9     5.6     -------
========== 9 / 50 (eta: 727.6 / 4042.3 s) =========
------- 20.2    23.7    5.8     8.6     -------
========== 10 / 50 (eta: 785.9 / 3929.5 s) =========
-

# obj3_pole

In [3]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [4]:
I_START=0
N_DAT = 50-I_START

### test

In [5]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 15.8    15.8    7.1     6.5     -------
========== 1 / 50 (eta: 45.2 / 2262.2 s) =========
------- 105.1   27.0    5.8     9.0     -------
========== 2 / 50 (eta: 192.2 / 4804.3 s) =========
------- 9.4     15.6    5.6     8.6     -------
========== 3 / 50 (eta: 231.3 / 3855.5 s) =========
------- 6.9     36.3    6.9     15.5    -------
========== 4 / 50 (eta: 296.9 / 3711.7 s) =========
------- 76.5    6.2     15.4    19.9    -------
========== 5 / 50 (eta: 414.9 / 4149.0 s) =========
------- 74.2    104.6   65.8    6.4     -------
========== 6 / 50 (eta: 666.0 / 5549.8 s) =========
------- 105.6   104.8   21.1    18.9    -------
========== 7 / 50 (eta: 916.4 / 6545.7 s) =========
------- 18.1    10.8    12.7    8.9     -------
========== 8 / 50 (eta: 966.9 / 6043.3 s) =========
------- 49.3    7.1     28.5    32.7    -------
========== 9 / 50 (eta: 1084.5 / 6025.2 s) =========
------- 113.0   46.3    13.0    7.2     -------
========== 10 / 50 (eta: 1264.1 / 6320.4 s) ========

## Res - panda
```

```

## Res - indy7
```
============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 18.58           6.56            4.06            0.11            2               20             
Tool           : 6.96            6.12            4.0             0.18            3               20             
ToolReach      : 2.79            5.44            3.67            0.17            2               20             
Full           : 2.22            5.25            3.12            0.0             4               20             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 30.37           8.73            5.2             0.07            5               20             
Tool           : 9.77            5.27            5.47            0.07            5               20             
ToolReach      : 7.24            6.0             7.42            0.17            8               20             
Full           : 6.2             5.53            4.53            0.0             5               20             
 
============================== obj_3_pole      ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 45.17           8.36            5.07            0.21            6               20             
Tool           : 16.48           5.71            7.65            0.53            3               20             
ToolReach      : 18.59           6.23            10.46           0.69            7               20             
Full           : 11.28           5.29            5.14            0.14            6               20             
```